# 问题
- 如何获取hidden梯度  

In [81]:
import numpy as np
import pandas as pd

import torch
from torch import nn,optim

import os

# 基本介绍

In [5]:
data_dir = "E:/DATA/feedback-prize-effectiveness/"

In [10]:
df1 = pd.read_csv(data_dir+"train.csv")
print(df1.head(10))
print(df1.iloc[0]['discourse_text'])
#essay_id   txt文件名
#discourse_id   段落的id，应该没用

   discourse_id      essay_id  \
0  0013cc385424  007ACE74B050   
1  9704a709b505  007ACE74B050   
2  c22adee811b6  007ACE74B050   
3  a10d361e54e4  007ACE74B050   
4  db3e453ec4e2  007ACE74B050   
5  36a565e45db7  007ACE74B050   
6  fb65fe816ba3  007ACE74B050   
7  4e472e2584fa  007ACE74B050   
8  28a94d3ee425  007ACE74B050   
9  d226f06362f5  00944C693682   

                                      discourse_text        discourse_type  \
0  Hi, i'm Isaac, i'm going to be writing about h...                  Lead   
1  On my perspective, I think that the face is a ...              Position   
2  I think that the face is a natural landform be...                 Claim   
3  If life was on Mars, we would know by now. The...              Evidence   
4  People thought that the face was formed by ali...          Counterclaim   
5  though some say that life on Mars does exist, ...              Rebuttal   
6  It says in paragraph 7, on April 5, 1998, Mars...              Evidence   
7  Everyone 

In [9]:
df2 = pd.read_csv(data_dir+"test.csv")
print(df2.head())
print(df2.iloc[0]['discourse_text'])

   discourse_id      essay_id  \
0  a261b6e14276  D72CB1C11673   
1  5a88900e7dc1  D72CB1C11673   
2  9790d835736b  D72CB1C11673   
3  75ce6d68b67b  D72CB1C11673   
4  93578d946723  D72CB1C11673   

                                      discourse_text discourse_type  
0  Making choices in life can be very difficult. ...           Lead  
1  Seeking multiple opinions can help a person ma...       Position  
2                     it can decrease stress levels           Claim  
3             a great chance to learn something new           Claim  
4               can be very helpful and beneficial.           Claim  
Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. 


In [12]:
with open(data_dir+"/train/000E6DE9E817.txt") as f:
    print(f.read())
print("-------")
with open(data_dir+"/train/00B144412785.txt") as f:
    print(f.read())

Dear: Principal

I am arguing against the policy change because even though there are some children out there that really needs help with their academic work, that does not mean that only because they have a c average that would not let them enjoy their sports or other activities unless they've a B average.

Sometimes teachers or even principal needs to consider that we should give the help that any student should have. Also this may consider student self as steam. Meaning student would start to feel sad nervous, and not wanting to go to school because of the reason they have a low averages and they can not participate in other activities or sports. The fact that there are children that would want to enjoy many good things the school is actually giving it to them.

We would want to make changes as, "like to be a better person for a better tomorrow" This supports the idea of having have many good thoughts and incasing your work as much as possible. In some situation like arguing we shou

# rnn介绍

## 数据基本处理
1. 一般语言处理中对全部数据只会取常用的n个词，在此之外的词是不认识的，即先有一本字典
2. 用one-hot编码时，每个词都表示为长n的向量，其中只有一个值是1，其余全是0。比如字典的第i（0开始）个单词其编码中1值的下标是i

## embedding
1. one_hot编码稀疏，所以考虑用稠密向量表示词，比如只用长为m<n的向量表示n个单词，其中单词间关系可以体现在向量间关系中，比如$\vec{男}+\vec{国王}=\vec{皇帝}$
2. 可由embedding层实现这个，embedding记录了一个(n,m)的矩阵，每行都是一个单词的稠密向量，作用是one-hot编码的向量按照其1值的下标i访问这个矩阵第i行，取出这行向量作为新输入
3. 具体使用看下面代码说明
4. embedding可由自行训练出，也可预加载预训练参数。使用预训练参数时，冻结此层

## 初步结果
将batch_size\*len_sentences\*n变为batch_size\*len_sentences\*m

## rnn具体流程
1. 首先初始化hadden_input为全0
2. 对每个词，其都会和当前的hadden_input一起进入网络(cat或add)进行一步（卷积、tanh激活）运算，所得的输出作为新的hadden_input与下一个词的向量一起进入网络（cat或add）
3. cat（最后一维）的话，需要截断，或者另外卷积一次获得新hadden_input

In [98]:
# rnn
batch_size = 4
sentence_len = 20 #一句话20个词
words_num = 100 #字典记录了100个词
words_len = 8 #字典的每个词向量长8
X = torch.randint(0,words_num,[batch_size,sentence_len]) #注意输入不为one-hot，只是每个词的字典序号，比如[3,2,10]表示一句话。int
y = torch.as_tensor([[0.,1],[0,1],[1,0],[1,0]])
print(X)
class MyRNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_size = 8
        self.embedding_layer = nn.Embedding(words_num,words_len,padding_idx=0) #20*8矩阵,padding_idx为输入长度不够时填充的字典词序号
        self.liner_layer = nn.Linear(words_len, self.hidden_size)

        self.hidden_layer = nn.Linear(self.hidden_size,self.hidden_size) #rnn关键部分

        self.out_layer = nn.Linear(self.hidden_size*sentence_len,2)
        self.activation_layer = nn.Softmax(dim=-1) #dim=0表示a[i][j][k]按i方向的几个数一起算，这里有问题？？？？？？？？？？？？？？

    def __init_hidden__(self):
        return torch.zeros([batch_size,self.hidden_size])

    def forward(self,x):
        self.hidden = self.__init_hidden__()

        print(f"before embedding:{x.shape}") #[4, 20]
        x = self.embedding_layer(x)
        print(f"after embedding:{x.shape}") #[4, 20, 8]
        print("-------------")

        #接下来是liner层
        x = self.liner_layer(x)
        print(f"after liner:{x.shape}") #[4, 20, 128]
        print("-------------")

        #rnn部分
        # 为了更好计算，将数据x变形为为len_sencentces*batch_size*words_num
        # 即x[0]为各句子首单词
        x = x.transpose(0,1)
        print(f"brfore rnn:{x.shape}") #[20, 4, 128]
        for i in range(x.shape[0]):
            x[i] = self.hidden_layer(x[i]+self.hidden)
            self.hidden = x[i]
        
        #恢复
        x = x.transpose(0,1)
        print(f"after rnn:{x.shape}") #[4, 20, 128]
        print("-------------")

        #展开
        x = x.flatten(1)
        print(f"after flatten:{x.shape}") #[4, 20, 128]
        print("-------------")

        #分类器
        out = self.out_layer(x)
        out = self.activation_layer(out)
        print(f"out shape:{out.shape}") #[4, 2]

        return out

rnn = MyRNN()
out = rnn(X)

tensor([[74, 72, 87, 65, 42, 78, 98, 29, 50,  1, 21, 63, 28, 85, 37, 57, 10, 19,
         47, 63],
        [23, 56, 44, 54, 34, 34, 98, 26, 39, 10, 79, 98, 38, 79, 76, 99, 32, 53,
         27, 85],
        [70,  3,  1, 72, 90, 42, 70, 57, 59, 51, 58, 63, 88, 87, 58, 41,  1, 78,
          9, 57],
        [81, 56, 18, 23, 33, 71, 13, 17, 59, 81,  4, 96, 86, 91, 70, 90, 87, 60,
         56, 29]])
before embedding:torch.Size([4, 20])
after embedding:torch.Size([4, 20, 8])
-------------
after liner:torch.Size([4, 20, 8])
-------------
brfore rnn:torch.Size([20, 4, 8])
after rnn:torch.Size([4, 20, 8])
-------------
after flatten:torch.Size([4, 160])
-------------
out shape:torch.Size([4, 2])


## pytorch的rnn
- 首先就是输入是batch_size\*len_sencentces\*words_num  
所以Embedding算是预处理部分，如果需要训练则？？？？？？？？

In [ ]:
batch_size = 4
sentences_len = 20 #一句话20个词
words_num = 8 #每个词向量长8
#rnn = nn.RNN(
#            input_size=words_num,
#            #hidden_size，num_layers 都是对网络的设置，与输入数据无关，设置相对自由
#            hidden_size=32,     # rnn hidden unit
#            num_layers=1,       # number of rnn layer
#            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
#        )